In [1]:
from models.DistMult import DistMult
from utils.loaders import load_data
import pickle
import torch
import numpy as np
from sklearn.utils import shuffle

In [2]:
with open(r'C:\Users\odhra\Documents\graph_project\allosteric_reasonings\data\processed\triplets.pickle', 'rb') as handle:
    data = pickle.load(handle)

with open(r'C:\Users\odhra\Documents\graph_project\allosteric_reasonings\data\processed\triplet_lookup.pickle', 'rb') as handle:
    lookup = pickle.load(handle)

with open(r'C:\Users\odhra\Documents\graph_project\allosteric_reasonings\data\processed\ASD_dictionary.pickle', 'rb') as handle:
    ASD_dictionary = pickle.load(handle)

with open(r'C:\Users\odhra\Documents\graph_project\allosteric_reasonings\data\processed\Sparse_dictionary.pickle', 'rb') as handle:
    BCE_dictionary = pickle.load(handle)

entities = int(len(lookup)/2)

In [3]:
number_of_batches = 10
number_of_epochs = 30
x = shuffle(data)
test_data = x[:100] #just limit data to the first 100 rows

In [4]:
all_objects, all_relationships, all_subjects = np.split(test_data, 3, axis=1)
all_objects = torch.LongTensor(all_objects)
all_subjects = torch.squeeze(torch.LongTensor(all_subjects))
# all_relationships = torch.LongTensor([a+1 for a in all_relationships])
all_relationships = torch.LongTensor(all_relationships)

In [95]:
def get_torch(a):
    oof = BCE_dictionary[(a[0],a[1])]
    return((oof.todense()))

In [96]:
testo = np.hstack((objects[index], relationships[index]))

In [117]:
BCE = np.apply_along_axis(get_torch, axis= 1, arr = testo)

In [118]:
testy = torch.Tensor(BCE)

In [116]:
torch.max(pred)

tensor(31.9829, grad_fn=<MaxBackward1>)

In [110]:
loss2 = torch.nn.BCELoss()

In [111]:
loss2(pred, testy)

RuntimeError: all elements of input should be between 0 and 1

In [5]:
model = DistMult(num_entities = entities, embedding_dim=100, num_relations=4)
optimiser = torch.optim.Adam(model.parameters(), lr=0.01)
for epoch in range(number_of_epochs):

        model.train() #tells pytorch you are training rather than evaluating, so dropout happens

        objects, subjects, relationships  = load_data(test_data, number_of_batches)

        for index in range(number_of_batches):

            obj = torch.LongTensor(objects[index])
            rel = torch.LongTensor(relationships[index])
            subj = torch.squeeze(torch.LongTensor(subjects[index]))


            optimiser.zero_grad()
            pred = model.forward(e1 = obj, rel = rel)
            loss = model.loss(pred, subj)
            loss.backward()
            optimiser.step()

        model.eval()

        # print(model.emb_rel(torch.LongTensor([1])))

        outputs = model.forward(e1 = all_objects, rel = all_relationships)
        _, predicted = torch.max(outputs.data, 1)

        print('number of correct answers is') 
        print((predicted == all_subjects).sum().item())



number of correct answers is
0
number of correct answers is
0
number of correct answers is
0
number of correct answers is
0
number of correct answers is
3
number of correct answers is
8
number of correct answers is
15
number of correct answers is
34
number of correct answers is
56


KeyboardInterrupt: 

In [ ]:
model.eval()
pred = model.forward(e1 = torch.LongTensor(objects[2]), rel = torch.LongTensor(relationships[2]))
_, output = torch.max(pred.data, 1)
output

In [ ]:
model.emb_rel(torch.LongTensor([3]))

In [ ]:
outputs = model.forward(e1 = all_objects, rel = all_relationships)
_, predicted = torch.max(outputs.data, 1)
print(predicted)

In [ ]:
# best way to test your system is to train on 100 examples and tests on the 100 same examples - if your model doesn't overfit after a bundle of epochs there is a mistake. 
# knowledge base models take a long long time to converge. They can take 100s of epochs to converge. YOU SHOULD SHUFFLE YOUR DATA EVERY TIME!!! Don't have the same stuff in each batch. 
# learn to use the data loader for shuffling. Alternatively do it yourself.  
# with BCE you can do label smoothing. 

In [ ]:
model.zero_grad()

print('emb_grad')
print(model.emb_e.weight.grad)

loss = model.loss(pred, subj)
loss.backward()

print('emb_grad')
print(model.emb_e.weight.grad)